<a href="https://colab.research.google.com/github/restrepo/twitter/blob/main/classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análisis de sentimientos de Twitter con Python
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en
* https://towardsdatascience.com/my-absolute-go-to-for-sentiment-analysis-textblob-3ac3a11d524
* https://www.justintodata.com/twitter-sentiment-analysis-python/
* https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

### 1) Descargar tweets

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [110]:
import pandas as pd
pd.set_option('display.max_colwidth',500)

In [105]:
df=pd.read_json('https://raw.githubusercontent.com/restrepo/twitter/main/tweets_df.json')

In [106]:
df.shape

(5000, 10)

In [ ]:
df.drop_duplicates(subset='text').shape

(2384, 10)

### Cree un nuevo clasificador
Vamos 
https://textblob.readthedocs.io/en/dev/classifiers.html#classifiers

* A favor del Paro → `'fav'`
* En contra del Paro→ `'con'`
* Informativo or neutro → `'neu'`
* Spam → `'spa'`

Para crear los conjutos de datos para training y para test debemos clasisificar un conjunto suficientemente grande de textos de tweets en las dos categorias en números similares para cada una de las dos categorias

Crearemos un algoritmo que nos permite clasificar algunos tweets del DataFrame en una nueva columna `'label'`, la cual tendrá un valor nulo cuando el tweet no este clasificado. Una vez un tweet sea clasificado, se añadirá a una lista de similaridad y sólo se clasificarán nuevos tweets que no sean similares a los previamente clasificados.  Para ello se usará el método `extractOne` del módulo `process` de `fuzzywuzzy` con scorer `fuzz.ratio` basado en la distancia Levenshtein entre dos textos.

In [ ]:
from fuzzywuzzy import process
from fuzzywuzzy import  fuzz

In [ ]:
n_train=50
n_test=75
n_traintest=n_train+n_test
df['label']=None
ii=0
similarity=[]
df=df.sample(df.shape[0])
for i in df.index:
    print("="*80)
    tweet=df.loc[i,'text']
    if similarity:
        if process.extractOne(tweet,similarity,scorer=fuzz.ratio)[-1]<90:
            similarity.append(tweet)
            posneg=input(f"{tweet}: https://twitter.com/twitter/status/{df.loc[i,'id']}\n → f/c/n/s para 'a favor'/'en contra'/'neutro'/'spam' or <Enter> para continuar:\n")
            if posneg=='c':
                posneg='con'
            elif posneg=='f':
                posneg='fav'
            elif posneg=='n':
                posneg='neu'
            elif posneg=='s':
                posneg='spa'
            else:
                continue
            df.loc[i,'label']=posneg
            ii=ii+1
    else:
        similarity.append(tweet)

    if ii==n_traintest:
        break

Select the train/test filtered DataFrame

In [ ]:
tmp=df[~df['label'].isna()].reset_index(drop=True)

In [ ]:
tmp.shape

(143, 11)

In [ ]:
tmp[:3]

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,id,original_id,label
0,Sandra Suarez - opiniones personales,,,False,2021-05-22 10:16:48,"RT @CapitnColombia: Por su presente, por su futuro, Construiremos un nuevo país. PRIMERA LINEA Y CAPITAN COLOMBIA!. \n[ 📸Diego Cortes ]\n#par…",None,Twitter for Android,1396047383784529920,1.395886e+18,fav
1,@marioperico4,,abajotwitter,False,2021-05-22 10:16:41,RT @martinquinco: @jflafaurie @UNPColombia Desbloqueo de vías ya!\n#NoMasParo \n#NoMasBloqueos \n#YoApoyoALaFuerzaPublica,"[NoMasParo, NoMasBloqueos, YoApoyoALaFuerzaPublica]",Twitter for Android,1396047356123041792,1.395756e+18,con
2,Mauro Gabriel Escalona Quiroz,,"Antiuribista 100% y 24/7, por que Colombia merece vivir en paz. Venceremos al tirano!!",False,2021-05-22 10:16:36,RT @Contagioradio1: #Bogotá Chicos de la guardia comunitaria primera línea en el portal de la resistencia protegiendo a la comunidad #ParoN…,[Bogotá],Twitter for Android,1396047336439259136,1.395937e+18,fav


In [ ]:
tmp.shape

(143, 11)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
stop_words=['rt',':']+stopwords.words('spanish')
def preprocess_tweet_text(tweet):
    tweet=tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, re.UNICODE ,flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    #tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = tweet.replace("  "," ").split()
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    #ps = PorterStemmer()
    #stemmed_words = [ps.stem(w) for w in filtered_words]
    #lemmatizer = WordNetLemmatizer()
    #lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/restrepo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tweet=tmp.loc[0,'text']

In [ ]:
preprocess_tweet_text(tweet)

'presente, futuro, construiremos nuevo país. primera linea capitan colombia!. [ 📸diego cortes ] par…'

In [ ]:
UNNNECESSARY=True
if not UNNNECESSARY:
    tmp['text']=tmp['text'].apply(preprocess_tweet_text)

In [50]:
RELOAD=True
if RELOAD:
    train=pd.read_json('train.json')
    test=pd.read_json('test.json')
else:
    train=tmp[:n_train][['text','label']].reset_index(drop=True)
    test=tmp[n_train:][['text','label']].reset_index(drop=True)

In [51]:
from textblob.classifiers import NaiveBayesClassifier

In [52]:
import pandas as pd

In [53]:
#

In [54]:
RETRAIN=False
if RETRAIN:
    tmp=pd.read_json('/home/restrepo/Downloads/tmp.json')
    n_train=50
    train=train.append(tmp[:n_train]).reset_index(drop=True)
    test =test.append(tmp[n_train:]).reset_index(drop=True)

In [55]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /home/restrepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
cl = NaiveBayesClassifier(  
    [ (d.get('text'),d.get('label')) for d in train.to_dict(orient='records')]  )

In [ ]:
cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )

In [57]:
SAVE=False
if SAVE:
    train.to_json('train.json',orient='records')
    test.to_json('train.json',orient='records')

In [58]:
fulltest=test.copy()
fulltest['test']=fulltest['text'].apply(cl.classify)

In [59]:
fulltest[['text','label','test']][:3]

,text,label,test
0,RT @FabioCardozoM: Como asumirá usted @JorgeIv...,con,con
1,RT @TatyMunozO: Los bloqueos afectan al pueblo...,con,con
2,RT @arturo2driguez: Compartan por favor sus ca...,fav,fav


In [67]:
tmp=train.append(test).reset_index(drop=True)

In [69]:
list()[10, 30, 50, 70, 90, 110, 

[10, 30, 50, 70, 90, 110, 130]

In [100]:
#n_train=50 Seems O.K
train=pd.read_json('train.json')
test=pd.read_json('test.json')
tmp=train.append(test).reset_index(drop=True)
n_try=100
clmax=0
for i in range(n_try):
    print('='*80)
    if i>0:
        tmp=tmp.sample(frac=1,random_state=i)
    for n_train in [50]:#range(20,100,5):
        train=tmp[n_train:]
        test =tmp[:n_train].reset_index(drop=True)
        cl = NaiveBayesClassifier(  
            [ (d.get('text'),d.get('label')) for d in train.to_dict(orient='records')]  )
        cln=cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )
        if cln>clmax:
            imax=i
            clmax=cln
            if i==45:
                break
    if i==45:
        break
        #print(i,n_train, cln, imax,clmax )
        

In [101]:
cln

0.94

In [95]:
#n_train=50 Seems O.K
train=pd.read_json('train.json')
test=pd.read_json('test.json')
tmp=train.append(test).reset_index(drop=True)
n_try=50
clmax=0
for i in [45]:#range(n_try):
    print('='*80)
    if i>0:
        tmp=tmp.sample(frac=1,random_state=i)
    for n_train in [50]:#range(20,100,5):
        train=tmp[n_train:]
        test =tmp[:n_train].reset_index(drop=True)
        cl = NaiveBayesClassifier(  
            [ (d.get('text'),d.get('label')) for d in train.to_dict(orient='records')]  )
        cln=cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )
        if cln>clmax:
            imax=i
            clmax=cln
        print(i,n_train, cln, imax,clmax )

45 50 0.88 45 0.88


In [60]:
cl.accuracy( [ (d.get('text'),d.get('label')) for d in test.to_dict(orient='records')] )

0.7551020408163265

In [102]:
fulltest['prob']=fulltest['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [103]:
fulltest

,text,label,test,prob
0,RT @FabioCardozoM: Como asumirá usted @JorgeIv...,con,con,1.00
1,RT @TatyMunozO: Los bloqueos afectan al pueblo...,con,con,1.00
2,RT @arturo2driguez: Compartan por favor sus ca...,fav,fav,1.00
3,Conozca el mapa de la solidaridad caleña #Paro...,fav,con,1.00
4,RT @personeriacali: #AEstaHora Continuamos rea...,fav,con,1.00
5,#ParoNacional18M #SOSColombiaDDHH #ParoNacion...,fav,fav,1.00
6,#Colombia 🇨🇴 | Ciudadanos denunciaron este mar...,fav,fav,1.00
7,RT @personeriacali: #AEstaHora las familias de...,con,con,1.00
8,#lawradio chévere que los periodistas tuvieran...,con,con,1.00
9,RT @UnivalleU: ¿Se van a perder la movilizació...,fav,con,1.00


Aplicar al DataFrame Completo

In [107]:
df['test']=df['text'].apply(cl.classify)
df['prob']=df['text'].apply(lambda t:   
                                cl.prob_classify( t ).prob(   cl.classify( t  )  ) ).round(2)

In [108]:
dfp=df.drop_duplicates('text').reset_index(drop=True)

In [111]:
dfp[['text','test','prob']].sample(10)

,text,test,prob
413,RT @MelSemerenah: ☝🤔💭 No MAME!!! Sr.\nPues a QUÉ?? HUELE 🤣😂🤣😂🤣😂\n#NoMasParo #SiempreFloresMoreno #animals #Oaxaca #Mexico #MewSuppasit #happy…,con,1.00
315,RT @claricegp: Cómo mujeres con @dignidad_col reiteramos nuestro rechazo al caso de violencia ejercido contra una agente de la Policía Naci…,fav,0.96
223,"RT @CaracolRadio: #Cali La Policía reveló que uno de sus atacantes ellos se subió encima de ella, empezó a tocarle todo el cuerpo, besarla…",fav,0.64
96,"#Bogota Portal las Américas situación fue tensa en los enfrentamientos entre el #ESMAD y #manifedtantes #paronacional #Colombia en Bogotá-Colombia, D.C https://t.co/489LU34qFB",fav,1.00
574,RT @Villegasalejo: #Dato Está todo listo para instalar la mesa de negociación entre gobierno y comité del paro. Delegación de @IvanDuque tr…,fav,0.87
579,"Lo dije hace casi un mes, estamos afrontando la realidad verdadera de un #ParoNacional que nos lleva de cabeza al caos https://t.co/irJCcCeQRn",con,0.77
393,RT @Justiciaypazcol: #ImagenHoy\n379 desaparecidos #ParoNacional \n¿Dónde están? https://t.co/VaS9y4PxPv,fav,0.96
234,"RT @VCT_txt: Es posible que alguien de #DerechosHumanos se pueda desplazar hasta Almacén Éxito de #Calipso, en #cali. Urge presencia.\nSolo…",fav,0.55
631,"RT @Gabocolombia76: Ante la Crisis del neoliberalismo, la guerra y la corrupción que han conllevado al estallido juvenil y popular, y al ac…",fav,0.93
371,@IvanCepedaCast que pongan al hp ese de Molano en su tal protestodromo para que los colombianos podamos practicar con el los ddhh que tanto ejercen los que están bajo su mando #soscolombianosestanmatando #nosestanmatando #soscolombia #Colombia #paronacional #theyarekillingus tombos hps,con,0.97


In [112]:
dfp.groupby('test')['test'].count()

test
con    241
fav    396
Name: test, dtype: int64

Sin quitar RT

In [113]:
df.groupby('test')['test'].count()

test
con    1201
fav    3799
Name: test, dtype: int64

In [114]:
import pickle

In [115]:
f=open('classifier.pickle','wb')
pickle.dump(cl,f)
f.close()

In [116]:
f=open('classifier.pickle','rb')
CL=pickle.load(f)
f.close()